# Self-Driving Car Engineer Nanodegree

## Project: **Advanced Lane Finding** 

### 08.Pipeline_video

In [11]:
#importing some useful packages
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
%matplotlib inline

### Load Camera parameters for Pkl

In [12]:
import pickle
fh = open('Camera_parameters.pkl', 'rb')
mtx = pickle.load(fh)
dist = pickle.load(fh)
fh.close()

In [13]:
#Define Undistorting the image function
def undistort(img):
    #Undistorting the image
    return cv2.undistort(img, mtx, dist, None, mtx)

In [14]:
def unwarp(img):
    image_size = (img.shape[1], img.shape[0])
    src = np.float32([(585,460),
                  (700,460), 
                  (260,680), 
                  (1000,680)])

    dst = np.float32([(300,0),
                      (image_size[0]-300,0),
                      (300,image_size[1]),
                      (image_size[0]-300,image_size[1])])
    img_size = (img.shape[1], img.shape[0])
    # use cv2.getPerspectiveTransform() to get M, the transform matrix, and Minv, the inverse
    M = cv2.getPerspectiveTransform(src, dst)
    
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped, M, Minv

In [15]:
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
def abs_sobel_thresh(img, orient='y', sobel_kernel=5, thresh=(60, 255)):
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    sobelx = cv2.Sobel(gray_img, cv2.CV_64F, (orient=='x'), (orient=='y'), ksize= sobel_kernel)
    sobely = cv2.Sobel(gray_img, cv2.CV_64F, (orient=='x'), (orient=='y'), ksize= sobel_kernel)
    # 3) Take the absolute value of the derivative or gradient
    abs_sobelx = np.absolute(sobelx)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    #binary_output = np.copy(img) # Remove this line
    return binary_output

def HLS_S_threshold(img, threshold=(200, 255)):

    HLS_img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    HLS_S_img = HLS_img[:,:,2]
    
    HLS_S_img = HLS_S_img*(255/np.max(HLS_S_img))
    
    binary_output = np.zeros_like(HLS_S_img)
    
    binary_output[(HLS_S_img > threshold[0]) & (HLS_S_img <= threshold[1])] = 1

    return binary_output
def combine_binary(binary_image1, binary_image2): 
    combined_binary = np.zeros_like(binary_image2)
    combined_binary[(binary_image1 == 1) | (binary_image2 == 1)] = 1
    return combined_binary


In [16]:
def find_lane_pixels(binary_warped):
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 20

    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty

def measure_curvature_pixels_in_realworld(combined_binary, ploty, leftx, lefty, rightx, righty, left_fit, right_fit):
    # meters per pixel in y dimension
    ym_per_pix = 30/720 
    # meters per pixel in x dimension
    xm_per_pix = 3.7/680 
    # Reverse to match top-to-bottom in y
    leftx = leftx[::-1]  
    # Reverse to match top-to-bottom in y
    rightx = rightx[::-1]  
    

    left_lane = np.mean(left_fit[0]*combined_binary[0]**2 + left_fit[1]*combined_binary[0] + left_fit[2])

    right_lane = np.mean(right_fit[0]*combined_binary[0]**2 + right_fit[1]*combined_binary[0] + right_fit[2])

    car_position = combined_binary.shape[1]/2

    center_lane_point = (left_lane + right_lane) /2

    center_miss = (car_position - center_lane_point) * xm_per_pix

    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)

    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    curverad = (left_curverad+right_curverad)/2
    
    return curverad, center_miss

def fit_polynomial(binary_warped):
    # HYPERPARAMETERS
    # Set the width of the windows +/- margin
    margin = 100
    # Find our lane pixels first
    leftx, lefty, rightx, righty= find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    curverad, center_miss = measure_curvature_pixels_in_realworld(binary_warped, ploty, leftx, lefty, rightx, righty, left_fit, right_fit)
    
    return curverad, center_miss, left_fit, right_fit

In [17]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fit, right_fit, left_fitx, right_fitx, ploty

def search_around_poly(binary_warped, left_fit, right_fit):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) &\
                    (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) &\
                    (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # Fit new polynomials
    left_fit, right_fit, left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    if (len(left_fit)==0) and (len(right_fit)==0):
        curverad, center_miss = measure_curvature_pixels_in_realworld(ploty, leftx, lefty, rightx, righty, left_fit, right_fit)
    else:
        pass
    ## End visualization steps ##
    return curverad, center_miss, left_fit, right_fit

In [18]:
def info_image(original_img, combined_binary, left_fit, right_fit, Minv, curverad, center_miss):

    ploty = np.linspace(0, combined_binary.shape[0]-1, combined_binary.shape[0])
    
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    new_image = np.copy(original_img)

    window_img = np.zeros((combined_binary.shape[0],combined_binary.shape[1],3), np.uint8)

    left_line_pts = np.array([np.transpose(np.vstack([left_fitx, ploty]))])

    right_line_pts = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])

    line_pts = np.hstack((left_line_pts, right_line_pts))

    cv2.fillPoly(window_img, np.int_([line_pts]), (0,255, 0))
    cv2.polylines(window_img, np.int32([left_line_pts]), color=(255,0,0), thickness=50, isClosed=False)
    cv2.polylines(window_img, np.int32([right_line_pts]), color=(0,0,255), thickness=50, isClosed=False)

    diswarp_lane_image = cv2.warpPerspective(window_img, Minv, (combined_binary.shape[1], combined_binary.shape[0]))

    # Combine the result with the original image
    result = cv2.addWeighted(new_image, 1, diswarp_lane_image, 0.3, 0)

    font = cv2.FONT_HERSHEY_PLAIN
    text= "CurveRadius: " + "{:03.2f}".format(curverad) + " Meter"

    cv2.putText(result, text, (30,50), font, 1.5, (255,255,255), 3, cv2.LINE_AA)

    message = ''
    if center_miss > 0:
        text = "Car is right of center: " "{:03.2f}".format(center_miss) + " Meter" 
    elif center_miss < 0:
        text = "Car is left  of center: " "{:03.2f}".format(abs(center_miss)) + " Meter" 

    cv2.putText(result, text, (30,100), font, 1.5, (255,255,255), 3, cv2.LINE_AA)
    
    return result

In [19]:
def process_image(original_image):    
    global left_fit_prev
    global right_fit_prev
    alpha = 0.001
    undistort_image = undistort(original_image)
    
    unwarp_image, M, Minv = unwarp(undistort_image)

    blur_image = gaussian_blur(unwarp_image, kernel_size=7)
    
    binary_image1 = abs_sobel_thresh(blur_image,orient='x',thresh=(60, 255))
    
    binary_image2 = HLS_S_threshold(blur_image, threshold=(160, 190))
    
    combined_binary = combine_binary(binary_image1, binary_image2)
    if left_fit_prev is None and right_fit_prev is None:
        curverad, center_miss, left_fit, right_fit= fit_polynomial(combined_binary)
    else:
        curverad, center_miss, left_fit, right_fit= search_around_poly(combined_binary, left_fit_prev, right_fit_prev)
    if not(left_fit_prev is None or right_fit_prev is None):
        left_fit = left_fit*alpha + left_fit_prev*(1-alpha)
        right_fit = right_fit*alpha + right_fit_prev*(1-alpha)
    drawed_image = info_image(original_image, combined_binary, left_fit, right_fit, Minv,curverad, center_miss)
    if left_fit_prev is None or right_fit_prev is None:
        left_fit_prev = left_fit_prev
        right_fit_prev = right_fit_prev
    else:
        curverad_prev = curverad
        center_miss_prev = center_miss
        left_fit_prev = left_fit
        right_fit_prev = right_fit
    return drawed_image

In [20]:
outputvideo = 'Output_project_video.mp4'
left_fit_prev = None
right_fit_prev = None
clip = VideoFileClip('project_video.mp4')
##clip = VideoFileClip('project_video.mp4').subclip(0,5)
outputvideo_clip = clip.fl_image(process_image)

%time outputvideo_clip.write_videofile(outputvideo, audio=False)

[MoviePy] >>>> Building video Output_project_video.mp4
[MoviePy] Writing video Output_project_video.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [02:23<00:00,  8.75it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: Output_project_video.mp4 

Wall time: 2min 24s


In [21]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(outputvideo))